In [1]:
import pandas as pd
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.chart import BarChart, Reference
import string
import os

class ExcelAutomator:
    def __init__(self, file_name):
        self.file_name = file_name
        print(file_name)
        self.month_and_extension = self.extract_month_and_extension()
        print(self.month_and_extension)

    def extract_month_and_extension(self):
        """Extracts the month and extension from file name."""
        return self.file_name.split('_')[1]
    
    def read_excel(self):
        """Reads the Excel file."""
        excel_df = pd.read_excel(self.file_name)
        return excel_df

    def create_pivot_table(self):
        """Creates a pivot table from the Excel data."""
        excel_file = self.read_excel()
        return excel_file.pivot_table(index='Gender', columns='Product line', values='Total', aggfunc='sum').round(0)
 
    def save_report_to_excel(self, report_table, output_dir='/'):
        """Saves the pivot table to an Excel file."""
        report_table.to_excel(f'report_{self.month_and_extension}', sheet_name='Report', startrow=4)

    def load_workbook(self):
        """Loads workbook"""
        return load_workbook(f'report_{self.month_and_extension}')

    def format_report(self, sheet):
        """Formats the report."""
        month_name = self.month_and_extension.split('.')[0]
        print(f'Month Name {month_name}')
        sheet['A1'] = 'Sales Report'
        sheet['A2'] = month_name.title()
        sheet['A1'].font = Font('Arial', bold=True, size=20)
        sheet['A2'].font = Font('Arial', bold=True, size=10)

    def apply_formulas(self, sheet, min_column, max_column, min_row, max_row):
        """Applies formulas to the sheet."""
        alphabet = list(string.ascii_uppercase)
        excel_alphabet = alphabet[0:max_column]
        for i in excel_alphabet:
            if i != 'A':
                sheet[f'{i}{max_row + 1}'] = f'=SUM({i}{min_row + 1}:{i}{max_row})'
                sheet[f'{i}{max_row + 1}'].style = 'Currency'
                sheet[f'{excel_alphabet[0]}{max_row + 1}'] = 'Total'

    def automate_excel(self):
        """Automates Excel report generation"""
        report_table = self.create_pivot_table()
        self.save_report_to_excel(report_table)
        wb = self.load_workbook()
        sheet = wb['Report']
        min_column = wb.active.min_column
        max_column = wb.active.max_column
        min_row = wb.active.min_row
        max_row = wb.active.max_row
        self.apply_formulas(sheet, min_column, max_column, min_row, max_row)
        self.add_chart_to_sheet(sheet, min_column, max_column, min_row, max_row)
        self.format_report(sheet)
        wb.save(f'report_{self.month_and_extension}')
        print(report_table)

    def add_chart_to_sheet(self, sheet, min_column, max_column, min_row, max_row):
        """Adds bar chart to the sheet"""
        barChart = BarChart()
        data = Reference(sheet, min_col=min_column+1, max_col=max_column, min_row=min_row,max_row=max_row) #including headers
        categories = Reference(sheet, min_col=min_column, max_col=min_column, min_row=min_row+1, max_row=max_row) #not including headers
        barChart.add_data(data, titles_from_data=True)      #adding data and categories
        barChart.set_categories(categories)
        sheet.add_chart(barChart, "B12")      #adding location chart
        barChart.title = 'Sales by Product line'
        barChart.style = 2         #choosing the chart style
        barChart.width = 20
        barChart.height = 10
        barChart.x_axis.title = 'Categories'
        barChart.y_axis.title = 'Sales'
        barChart.x_axis.delete = False
        barChart.y_axis.delete = False
        barChart.legend.position = 'r'
        barChart.legend.overlay = False
        barChart.gapWidth = 150


# calls class excelautomator
file_name = 'sales_2021.xlsx'
automator = ExcelAutomator(file_name)
automator.automate_excel()

sales_2021.xlsx
2021.xlsx
Month Name 2021
Product line  Electronic accessories  Fashion accessories  Food and beverages  \
Gender                                                                          
Female                       27102.0              30437.0             33171.0   
Male                         27236.0              23868.0             22974.0   

Product line  Health and beauty  Home and lifestyle  Sports and travel  
Gender                                                                  
Female                  18561.0             30037.0            28575.0  
Male                    30633.0             23825.0            26548.0  
